# 🌟 D²NN v19 - CONSOLIDADO
## **Digital Twin for Generative Optical Computing**

---

### ✨ Combina o melhor da V8 e V18:
- ✅ **Visualização V8**: GIFs de alta qualidade com treinamento energy scaling
- ✅ **Física V18**: Medições em mW, DOE export 16-bit, specs de fabricação
- ✅ **Análise completa**: SSIM/PSNR/MSE + Eficiência + SNR

---

### 📚 Contexto para Estudantes de IA

| Conceito de IA | Equivalente Óptico |
|:---------------|:-------------------|
| Neurônio | Pixel do modulador espacial de luz (SLM) |
| Pesos | Espessura/fase da máscara difrativa |
| Forward Pass | Propagação física da luz (difração) |
| Backpropagation | Gradiente via método adjunto |
| Inferência | Passagem de luz (~ns vs ~ms em GPUs) |

---

### ⚡ CONCEITO FÍSICO: Conservação de Energia

```
┌──────────────────────────────────────────────────────────────────┐
│  🔴 LASER HeNe (633nm) → Fonte de energia FINITA                │
├──────────────────────────────────────────────────────────────────┤
│                                                                  │
│  ENTRADA (Máscara):          SAÍDA (Sensor):                     │
│  ┌─────────┐                 ┌─────────┐                        │
│  │ BRANCO  │ = Luz passa     │ CLARO   │ = Energia concentrada  │
│  │ CINZA   │ = Luz parcial   │ CINZA   │ = Energia média        │
│  │ PRETO   │ = Luz bloqueada │ ESCURO  │ = Pouca energia        │
│  └─────────┘                 └─────────┘                        │
│                                                                  │
│  💡 PRINCÍPIO: Camadas DOE ATENUAM e REDISTRIBUEM energia       │
│                                                                  │
│  📊 O QUE CADA CAMADA FAZ:                                      │
│     T = sigmoid(pesos) → Transmissão ∈ [0,1]                    │
│     • T alto (~70%+) = luz passa (pouca modulação)              │
│     • T médio (~30-50%) = modulação + atenuação                 │
│     • T baixo (~10%) = forte atenuação (filtragem espacial)     │
│                                                                  │
│  ⚠️ ATENUAÇÃO ACUMULATIVA:                                      │
│     T_total = T1 × T2 × T3 × T4 × T5                            │
└──────────────────────────────────────────────────────────────────┘
```

**Referências:**
- Chen, S. et al. *Nature* 644, 903-910 (2025)
- Lin, X. et al. *Science* 361, 1004-1008 (2018)
- Goodman, J.W. Introduction to Fourier Optics (2005)

In [ ]:
# ══════════════════════════════════════════════════════════════════
# CÉLULA 1: IMPORTS + GPU
# ══════════════════════════════════════════════════════════════════
import subprocess, sys

def install_if_missing(package):
    try:
        __import__(package)
    except ImportError:
        print(f"Installing {package}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", package])

install_if_missing('imageio')
install_if_missing('scikit-image')

import os, time, base64, warnings, shutil
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import imageio
from torch.cuda.amp import autocast, GradScaler
from PIL import Image
from IPython.display import HTML, display
from io import BytesIO
from skimage.metrics import structural_similarity as ssim, peak_signal_noise_ratio as psnr
from datetime import datetime

warnings.filterwarnings('ignore')
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

if torch.cuda.is_available():
    torch.backends.cuda.matmul.allow_tf32 = True
    torch.backends.cudnn.allow_tf32 = True
    torch.backends.cudnn.benchmark = True
    print(f"🚀 GPU: {torch.cuda.get_device_name(0)} | {torch.cuda.get_device_properties(0).total_memory/1e9:.1f}GB | TF32: ON")
else:
    print("⚠️ CPU mode")

# Estrutura de pastas
FOLDERS = ['optical_fader', 'optical_mj']
for folder in FOLDERS:
    if os.path.exists(folder):
        shutil.rmtree(folder)
    os.makedirs(folder, exist_ok=True)

print("\n" + "═"*70)
print("🔬 D²NN v19 - CONSOLIDADO (V8 Visual + V18 Physics)")
print("═"*70)
print(f"📅 Start: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"📂 Folders: {FOLDERS}")
print("✅ Imports OK")

In [ ]:
# ══════════════════════════════════════════════════════════════════
# CÉLULA 2: PARÂMETROS FÍSICOS (V18 Enhanced + Phase/Amp Balance)
# ══════════════════════════════════════════════════════════════════

class PhysicalParams:
    """Parâmetros físicos calibrados para DOE real."""
    # Sistema Óptico
    grid_size = 256
    wavelength = 633e-9              # HeNe laser
    pixel_size = 20e-6               # 20µm SLM pixel
    n_substrate = 1.46               # SiO2

    # DOE
    num_layers = 5
    distances = (0.04, 0.04, 0.04, 0.04, 0.2)  # metros

    # Laser (V18)
    laser_power_mW = 20.0            # 20mW input
    beam_diameter_mm = 4.0           # 4mm beam

    # Fabricação (V18)
    phase_bits = 16                  # 16-bit lithography
    amp_bits = 8                     # 8-bit chrome deposition

    # ════════════════════════════════════════════════════════════════
    # 🎛️ HIPERPARÂMETRO: BALANÇO FASE × AMPLITUDE
    # ════════════════════════════════════════════════════════════════
    # Controla T_inicial = sigmoid(init_bias) → transmissão inicial
    # Valores negativos → Amplitude dominante (mais atenuação)
    # Valores positivos → Fase dominante (mais transmissão)
    init_bias = 1.0                  # 🔧 AJUSTE AQUI! Default: 1.0 (fase)

PARAMS = PhysicalParams()
K = 2 * np.pi / PARAMS.wavelength
plane_wave = torch.ones(PARAMS.grid_size, PARAMS.grid_size, dtype=torch.complex64, device=DEVICE)

# ════════════════════════════════════════════════════════════════════
# TABELA DINÂMICA: BALANÇO FASE × AMPLITUDE
# ════════════════════════════════════════════════════════════════════
def sigmoid(x): return 1 / (1 + np.exp(-x))

bias_options = [
    (-3.0, 'Amplitude extrema (absorção máxima)'),
    (-2.0, 'Amplitude dominante (forte atenuação)'),
    (-1.0, 'Amplitude + alguma fase'),
    ( 0.0, 'Híbrido equilibrado (50/50)'),
    ( 1.0, 'Fase + alguma amplitude'),
    ( 2.0, 'Fase dominante (luz passa, pouca atenuação)'),
    ( 3.0, 'Fase extrema (transmissão máxima)'),
]

print("\n" + "═"*78)
print("🎛️  TABELA DE BALANÇO: FASE × AMPLITUDE (init_bias)")
print("═"*78)
print(f"{'INIT_BIAS':^10} │ {'T_inicial':^10} │ {'Fase':^6} │ {'Amp':^6} │ {'Comportamento':<32}")
print("─"*78)

for bias, desc in bias_options:
    T = sigmoid(bias) * 100
    fase_pct = T
    amp_pct = 100 - T
    marker = ' ◀── SELECIONADO' if abs(bias - PARAMS.init_bias) < 0.01 else ''
    print(f"{bias:^+10.1f} │ {T:^9.0f}% │ {fase_pct:^5.0f}% │ {amp_pct:^5.0f}% │ {desc:<32}{marker}")

print("═"*78)
T_selected = sigmoid(PARAMS.init_bias) * 100
print(f"\n📊 CONFIGURAÇÃO ATUAL: init_bias = {PARAMS.init_bias:+.1f}")
print(f"   → Transmissão inicial: {T_selected:.0f}%")
print(f"   → Balanço: {T_selected:.0f}% Fase / {100-T_selected:.0f}% Amplitude")
if PARAMS.init_bias > 0:
    print(f"   → Tendência: DOE de FASE (eficiente, menos absorção)")
elif PARAMS.init_bias < 0:
    print(f"   → Tendência: DOE de AMPLITUDE (mais contraste, mais absorção)")
else:
    print(f"   → Tendência: HÍBRIDO (balanço neutro)")

print("\n" + "═"*70)
print("⚡ CONFIGURAÇÃO DO SISTEMA ÓPTICO")
print("═"*70)
print(f"  λ = {PARAMS.wavelength*1e9:.0f}nm (HeNe laser vermelho)")
print(f"  Substrate: SiO₂ (n={PARAMS.n_substrate})")
print(f"  Grid = {PARAMS.grid_size}×{PARAMS.grid_size} pixels")
print(f"  Pixel = {PARAMS.pixel_size*1e6:.0f}µm")
print(f"  Camadas = {PARAMS.num_layers}")
print(f"  Distâncias = {[f'{d*100:.0f}cm' for d in PARAMS.distances]}")
print(f"  Laser: {PARAMS.laser_power_mW}mW @ {PARAMS.beam_diameter_mm}mm")
print(f"  Fabrication: {PARAMS.phase_bits}-bit phase / {PARAMS.amp_bits}-bit amplitude")
print(f"  Init Bias: {PARAMS.init_bias:+.1f} (T₀={T_selected:.0f}%)")
print("═"*70)

In [ ]:
# ══════════════════════════════════════════════════════════════════
# CÉLULA 3: MODELO D2NN (V8 Style - Melhor para visualização)
# ══════════════════════════════════════════════════════════════════

class D2NN(nn.Module):
    """
    Rede Neural Difrativa de 5 camadas.

    Cada camada aplica: field = field * T * exp(propagação)
    onde T = sigmoid(pesos) é a transmissão [0,1]

    IMPORTANTE: A transmissão T ATENUA a luz progressivamente.
    Se T_médio = 0.35, então ~65% da energia é bloqueada por camada.
    Com 5 camadas: T_total ≈ 0.35^5 = 0.5% da energia original.

    INIT_BIAS: Controla balanço inicial fase/amplitude
    - bias > 0: T alto → DOE de fase (menos absorção)
    - bias < 0: T baixo → DOE de amplitude (mais absorção)
    """

    def __init__(self, params):
        super().__init__()
        self.params = params
        # Inicialização com bias para controlar balanço fase/amplitude
        self.layers = nn.ParameterList([
            nn.Parameter(torch.randn(params.grid_size, params.grid_size, device=DEVICE) * 0.1 + params.init_bias)
            for _ in range(params.num_layers)
        ])
        self._precompute_kernels()

        # Log da inicialização
        T_init = 1 / (1 + np.exp(-params.init_bias))
        print(f"   🔧 Init bias: {params.init_bias:+.1f} → T₀={T_init*100:.0f}% ({T_init*100:.0f}% fase / {(1-T_init)*100:.0f}% amplitude)")

    def _precompute_kernels(self):
        N, dx = self.params.grid_size, self.params.pixel_size
        fx = torch.fft.fftshift(torch.fft.fftfreq(N, d=dx, device=DEVICE))
        fy = torch.fft.fftshift(torch.fft.fftfreq(N, d=dx, device=DEVICE))
        FX, FY = torch.meshgrid(fx, fy, indexing='xy')
        self.freq_sq = (2*np.pi)**2 * (FX**2 + FY**2)
        self.kernels = {}
        for z in set(self.params.distances):
            kz = torch.sqrt(torch.clamp(K**2 - self.freq_sq, min=0).to(torch.complex64))
            self.kernels[z] = torch.exp(1j * kz * z)

    def forward(self, x):
        field = x.to(torch.complex64)
        for i, w in enumerate(self.layers):
            T = torch.sigmoid(w)
            field = field * T.to(torch.complex64)
            z = self.params.distances[i]
            field = torch.fft.ifft2(torch.fft.ifftshift(
                torch.fft.fftshift(torch.fft.fft2(field)) * self.kernels[z]
            ))
        return torch.abs(field)**2

    def forward_with_intermediates(self, x):
        intermediates = {'input': x.detach().cpu()}
        field = x.to(torch.complex64)
        for i, w in enumerate(self.layers):
            T = torch.sigmoid(w)
            field = field * T.to(torch.complex64)
            z = self.params.distances[i]
            field = torch.fft.ifft2(torch.fft.ifftshift(
                torch.fft.fftshift(torch.fft.fft2(field)) * self.kernels[z]
            ))
            intermediates[f'L{i+1}'] = field.detach().cpu()
        intermediates['output'] = torch.abs(field)**2
        return intermediates

    def get_transmissions(self):
        """Retorna transmissão média de cada camada."""
        with torch.no_grad():
            return [torch.sigmoid(w).mean().item() for w in self.layers]

    def get_transmission_maps(self):
        """Retorna mapas de transmissão (onde luz passa em cada camada)."""
        with torch.no_grad():
            return [torch.sigmoid(w).cpu().numpy() for w in self.layers]

    def get_theoretical_transmission(self):
        """Calcula transmissão teórica acumulada (produto das médias)."""
        T = self.get_transmissions()
        return np.prod(T)

    def get_energy_flow(self, x):
        """Calcula fluxo de energia através de cada estágio."""
        with torch.no_grad():
            energies = []
            field = x.to(torch.complex64)
            energies.append((torch.abs(field)**2).sum().item())  # Input

            for i, w in enumerate(self.layers):
                T = torch.sigmoid(w)
                field = field * T.to(torch.complex64)
                # Energia após transmissão (antes da propagação)
                energies.append((torch.abs(field)**2).sum().item())
                z = self.params.distances[i]
                field = torch.fft.ifft2(torch.fft.ifftshift(
                    torch.fft.fftshift(torch.fft.fft2(field)) * self.kernels[z]
                ))

            return energies

print("✅ D2NN Model OK")

In [ ]:
# ══════════════════════════════════════════════════════════════════
# CÉLULA 4: PHYSICAL POWER ANALYZER (V18 Feature)
# ══════════════════════════════════════════════════════════════════

class PhysicalPowerAnalyzer:
    """
    Calcula potência REAL em mW através do sistema.
    Exporta DOEs em formato de fabricação.
    """
    def __init__(self, model, params):
        self.model = model
        self.params = params

    def calculate_power_mW(self, input_mask):
        """
        Calcula fluxo de potência em mW.

        Returns:
            dict: {
                'input_mW': potência entrada,
                'layer_X_mW': potência após cada camada,
                'output_mW': potência detector,
                'efficiency_%': eficiência total
            }
        """
        with torch.no_grad():
            powers = {}

            # Potência de entrada
            P_input = self.params.laser_power_mW
            powers['input_mW'] = P_input

            # Simular propagação
            field = input_mask.to(torch.complex64)

            for i, w in enumerate(self.model.layers):
                # Transmissão
                T = torch.sigmoid(w)
                field = field * T.to(torch.complex64)

                # Potência após transmissão
                intensity = torch.abs(field)**2
                P_layer = intensity.mean().item() * P_input
                powers[f'L{i+1}_mW'] = P_layer
                powers[f'L{i+1}_T'] = T.mean().item()

                # Propagação
                z = self.params.distances[i]
                field = torch.fft.ifft2(torch.fft.ifftshift(
                    torch.fft.fftshift(torch.fft.fft2(field)) * self.model.kernels[z]
                ))

            # Potência final no detector
            output_intensity = torch.abs(field)**2
            P_output = output_intensity.mean().item() * P_input
            powers['output_mW'] = P_output

            # Eficiência
            efficiency = (P_output / P_input) * 100
            powers['efficiency_%'] = efficiency

            # SNR estimado (shot noise limited)
            if P_output > 0:
                E_photon = 6.626e-34 * 3e8 / self.params.wavelength
                photon_rate = (P_output * 1e-3) / E_photon
                N_photons = photon_rate * 1e-3  # 1ms integration
                SNR = np.sqrt(N_photons)
                powers['SNR_1ms'] = SNR
            else:
                powers['SNR_1ms'] = 0

        return powers

    def print_power_summary(self, input_mask, title="Power Analysis"):
        """Imprime resumo de potência."""
        powers = self.calculate_power_mW(input_mask)

        print(f"\n{'─'*70}")
        print(f"⚡ {title}")
        print(f"{'─'*70}")
        print(f" 📥 Input:    {powers['input_mW']:.2f} mW")

        for i in range(self.params.num_layers):
            P = powers[f'L{i+1}_mW']
            T = powers[f'L{i+1}_T']
            print(f" 🔬 L{i+1}:       {P:.4f} mW (T={T*100:.1f}%)")

        print(f" 📊 Output:   {powers['output_mW']:.4f} mW")
        print(f" 📈 Efficiency: {powers['efficiency_%']:.3f}%")
        print(f" 📷 SNR (1ms):  {powers['SNR_1ms']:.1f}")
        print(f"{'─'*70}")

        return powers

    def export_does_16bit(self, output_dir, phase_name="phase"):
        """
        Exporta DOEs em 16-bit TIFF (fabricação).
        """
        os.makedirs(output_dir, exist_ok=True)

        print(f"\n📦 Exportando DOEs 16-bit: {output_dir}")

        with torch.no_grad():
            for i, w in enumerate(self.model.layers):
                # Transmissão
                T = torch.sigmoid(w).detach().cpu().numpy()

                # Calcular altura de fase equivalente
                # Para um DOE de amplitude, simulamos a fase a partir do peso
                phi = w.detach().cpu().numpy()
                phi_wrapped = np.mod(phi, 2*np.pi)

                # Conversão: h = φ × λ / (2π × (n-1))
                height_m = phi_wrapped * self.params.wavelength / \
                           (2*np.pi * (self.params.n_substrate - 1))
                height_um = height_m * 1e6

                # Salvar NPY
                np.save(f'{output_dir}/{phase_name}_L{i+1}_height_um.npy', height_um)
                np.save(f'{output_dir}/{phase_name}_L{i+1}_transmission.npy', T)

                # Salvar 16-bit TIFF (litografia)
                height_norm = (height_um - height_um.min()) / \
                             (height_um.max() - height_um.min() + 1e-10) * 65535
                height_16bit = height_norm.astype(np.uint16)
                Image.fromarray(height_16bit).save(
                    f'{output_dir}/{phase_name}_L{i+1}_phase_16bit.tiff'
                )

                # Salvar 8-bit PNG (transmissão)
                T_8bit = (T * 255).astype(np.uint8)
                Image.fromarray(T_8bit).save(
                    f'{output_dir}/{phase_name}_L{i+1}_amp_8bit.png'
                )

                print(f"   ✅ L{i+1}: h={height_um.min():.3f}-{height_um.max():.3f}µm, "
                      f"T={T.mean()*100:.1f}%")

        # Relatório de fabricação
        self._write_fabrication_specs(output_dir, phase_name)
        print(f"   📄 Specs: {phase_name}_fabrication_specs.txt")
        print("─"*70)

    def _write_fabrication_specs(self, output_dir, phase_name):
        """Escreve relatório de especificações de fabricação."""
        with open(f'{output_dir}/{phase_name}_fabrication_specs.txt', 'w', encoding='utf-8') as f:
            f.write("="*70 + "\n")
            f.write(f"DOE FABRICATION SPECS - {phase_name.upper()}\n")
            f.write("="*70 + "\n\n")
            f.write(f"System:\n")
            f.write(f"  Wavelength: {self.params.wavelength*1e9:.0f} nm\n")
            f.write(f"  Substrate: SiO2 (n={self.params.n_substrate})\n")
            f.write(f"  Grid: {self.params.grid_size}×{self.params.grid_size}\n")
            f.write(f"  Pixel: {self.params.pixel_size*1e6:.0f} µm\n")
            f.write(f"  Layers: {self.params.num_layers}\n")
            f.write(f"  Laser Power: {self.params.laser_power_mW} mW\n\n")

            for i, w in enumerate(self.model.layers):
                T = torch.sigmoid(w).detach().cpu().numpy()
                phi = w.detach().cpu().numpy()
                phi_wrapped = np.mod(phi, 2*np.pi)
                height_m = phi_wrapped * self.params.wavelength / \
                           (2*np.pi * (self.params.n_substrate - 1))
                height_um = height_m * 1e6

                f.write(f"Layer {i+1}:\n")
                f.write(f"  Phase Height: {height_um.min():.3f}-{height_um.max():.3f} µm\n")
                f.write(f"  Amplitude: T={T.mean()*100:.1f}% (σ={T.std()*100:.1f}%)\n\n")

print("✅ PhysicalPowerAnalyzer OK")

In [ ]:
# ══════════════════════════════════════════════════════════════════
# CÉLULA 5: GERADORES DE DADOS
# ══════════════════════════════════════════════════════════════════

def create_fader_mask(pos, N=256, only_track=False):
    mask = torch.ones(N, N, device=DEVICE)
    tw, b = int(0.12*N), max(1, int(0.01*N))
    mask[:, :b] = mask[:, tw-b:tw] = mask[:b, :tw] = mask[N-b:, :tw] = 0
    if only_track: return mask
    ss = int(0.1*N)
    step = (N - 2*b - ss) // 9
    y = b + pos * step
    mask[y:y+ss, b:tw-b] = 0
    return mask

def create_7segment(d, N=256):
    t = torch.zeros(N, N, device=DEVICE)
    cx, cy, sl, st = N//2, N//2, N//4, max(2, N//32)
    gap = sl//2 + st
    segs = {0:[0,1,2,3,4,5], 1:[1,2], 2:[0,1,6,4,3], 3:[0,1,6,2,3],
            4:[5,6,1,2], 5:[0,5,6,2,3], 6:[0,5,4,3,2,6], 7:[0,1,2],
            8:[0,1,2,3,4,5,6], 9:[0,1,2,3,5,6]}
    def draw(s):
        if s==0: t[cy-gap-st:cy-gap, cx-sl//2:cx+sl//2]=1
        elif s==1: t[cy-gap:cy, cx+sl//2-st:cx+sl//2]=1
        elif s==2: t[cy:cy+gap, cx+sl//2-st:cx+sl//2]=1
        elif s==3: t[cy+gap:cy+gap+st, cx-sl//2:cx+sl//2]=1
        elif s==4: t[cy:cy+gap, cx-sl//2:cx-sl//2+st]=1
        elif s==5: t[cy-gap:cy, cx-sl//2:cx-sl//2+st]=1
        elif s==6: t[cy-st//2:cy+st//2, cx-sl//2:cx+sl//2]=1
    for s in segs.get(d,[]): draw(s)
    return t

def create_fader_count(N=256):
    t = torch.zeros(N, N, device=DEVICE)
    cm = {
        'F':[[1,1,1,1],[1,0,0,0],[1,1,1,0],[1,0,0,0],[1,0,0,0]],
        'A':[[0,1,1,0],[1,0,0,1],[1,1,1,1],[1,0,0,1],[1,0,0,1]],
        'D':[[1,1,1,0],[1,0,0,1],[1,0,0,1],[1,0,0,1],[1,1,1,0]],
        'E':[[1,1,1,1],[1,0,0,0],[1,1,1,0],[1,0,0,0],[1,1,1,1]],
        'R':[[1,1,1,0],[1,0,0,1],[1,1,1,0],[1,0,1,0],[1,0,0,1]],
        'C':[[0,1,1,1],[1,0,0,0],[1,0,0,0],[1,0,0,0],[0,1,1,1]],
        'O':[[0,1,1,0],[1,0,0,1],[1,0,0,1],[1,0,0,1],[0,1,1,0]],
        'U':[[1,0,0,1],[1,0,0,1],[1,0,0,1],[1,0,0,1],[0,1,1,0]],
        'N':[[1,0,0,1],[1,1,0,1],[1,0,1,1],[1,0,0,1],[1,0,0,1]],
        'T':[[1,1,1,1],[0,1,1,0],[0,1,1,0],[0,1,1,0],[0,1,1,0]]
    }
    ch, cw, sp = int(N*0.045), int(N*0.035), max(2, int(N*0.015))
    for li, txt in enumerate(["FADER","COUNT"]):
        tw = len(txt)*(cw*4+sp*3)+(len(txt)-1)*cw
        cx = (N-tw)//2
        cy = int(N*0.22)+li*(ch*7)
        for c in txt:
            m = cm.get(c,[[0]*4]*5)
            for r in range(5):
                for co in range(4):
                    if m[r][co]:
                        t[cy+r*(ch+sp)//2:cy+r*(ch+sp)//2+ch, cx+co*(cw+sp):cx+co*(cw+sp)+cw]=1
            cx += cw*4+sp*3+cw
    return t

# Demonstração
print("\n" + "═"*60)
print("📊 BALANÇO DE ENERGIA: INPUT vs TARGET")
print("═"*60)
mask = create_fader_mask(5, PARAMS.grid_size)
target = create_7segment(5, PARAMS.grid_size)
E_in = mask.mean().item()
E_tgt = target.mean().item()
print(f"  INPUT:  {E_in*100:5.1f}% pixels brancos (energia disponível)")
print(f"  TARGET: {E_tgt*100:5.1f}% pixels iluminados (energia necessária)")
print(f"  RAZÃO:  {E_in/E_tgt:.0f}× mais energia que o necessário")
print(f"  💡 Tarefa: REDISTRIBUIR luz dispersa em padrão específico")
print("═"*60)

In [ ]:
# ══════════════════════════════════════════════════════════════════
# CÉLULA 6: TREINAMENTO FADER COUNT (V8 Style + V18 Metrics)
# ══════════════════════════════════════════════════════════════════

def train_fader(model, analyzer, epochs=3000, lr=0.02):
    """Treina Fader Count com energy scaling (V8) + medições mW (V18)."""
    opt = optim.Adam(model.parameters(), lr=lr)
    sch = optim.lr_scheduler.ReduceLROnPlateau(opt, 'min', 0.5, 100)
    loss_fn = nn.MSELoss()
    hist = []
    t0 = time.time()

    print("\n" + "═"*70)
    print("🔥 TREINAMENTO: FADER COUNT")
    print("═"*70)
    print("  ✅ Energy Scaling (V8): sc = target.mean() / output.mean()")
    print("  ✅ Weighted Loss: texto=2.5×")
    print(f"  ✅ Laser power: {PARAMS.laser_power_mW} mW")
    print("═"*70)

    for e in range(epochs):
        model.train()
        opt.zero_grad()
        tot = 0

        # Texto (peso 2.5×) com Energy Scaling
        m = create_fader_mask(0, PARAMS.grid_size, True)
        o = model(plane_wave * m)
        tg = create_fader_count(PARAMS.grid_size)
        sc = tg.mean() / (o.mean() + 1e-8)  # Energy Scaling V8!
        l = loss_fn(o * sc, tg) * 2.5
        l.backward(); tot += l.item()

        # Dígitos 0-9 com Energy Scaling
        for d in range(10):
            m = create_fader_mask(d, PARAMS.grid_size)
            o = model(plane_wave * m)
            tg = create_7segment(d, PARAMS.grid_size)
            sc = tg.mean() / (o.mean() + 1e-8)  # Energy Scaling V8!
            l = loss_fn(o * sc, tg)
            l.backward(); tot += l.item()

        opt.step(); sch.step(tot); hist.append(tot)

        if e % 500 == 0:
            T = model.get_transmissions()
            # Medições em mW (V18)
            test_input = plane_wave * create_fader_mask(5, PARAMS.grid_size)
            powers = analyzer.calculate_power_mW(test_input)
            print(f"\nEpoch {e}/{epochs}:")
            print(f"  Loss: {tot:.4f}")
            print(f"  T: [{T[0]:.2f},{T[1]:.2f},{T[2]:.2f},{T[3]:.2f},{T[4]:.2f}]")
            print(f"  Detector Power: {powers['output_mW']:.4f} mW")
            print(f"  Efficiency: {powers['efficiency_%']:.3f}%")
            print(f"  SNR (1ms): {powers['SNR_1ms']:.1f}")

            # Export DOEs desta etapa
            analyzer.export_does_16bit('optical_fader', f'epoch_{e:04d}')

    elapsed = time.time() - t0
    print(f"\n✅ Training complete: {elapsed:.0f}s | Loss: {hist[0]:.4f} → {hist[-1]:.4f}")
    return hist, elapsed

# Criar modelo e analisador
model_fader = D2NN(PARAMS).to(DEVICE)
analyzer_fader = PhysicalPowerAnalyzer(model_fader, PARAMS)

print(f"📦 Modelo: {sum(p.numel() for p in model_fader.parameters()):,} params")
history_fader, time_fader = train_fader(model_fader, analyzer_fader, epochs=3000, lr=0.02)

# Análise final
test_input = plane_wave * create_fader_mask(5, PARAMS.grid_size)
analyzer_fader.print_power_summary(test_input, "FADER FINAL ANALYSIS")

# Export DOEs finais
analyzer_fader.export_does_16bit('optical_fader', 'final')

# Salvar modelo
torch.save(model_fader.state_dict(), 'optical_fader/model_fader_v19.pth')
print("💾 Model saved: optical_fader/model_fader_v19.pth")

In [ ]:
# ══════════════════════════════════════════════════════════════════
# CÉLULA 7: ANÁLISE DE ENERGIA FADER (V8 Style)
# ══════════════════════════════════════════════════════════════════

print("\n" + "═"*65)
print("⚡ ANÁLISE DE ENERGIA: ATENUAÇÃO vs REDISTRIBUIÇÃO")
print("═"*65)

T_means = model_fader.get_transmissions()
T_theoretical = model_fader.get_theoretical_transmission()

print("\n📊 TRANSMISSÃO POR CAMADA:")
for i, t in enumerate(T_means):
    bar = "█" * int(t * 20) + "░" * (20 - int(t * 20))
    attn = (1 - t) * 100
    print(f"  L{i+1}: [{bar}] {t*100:5.1f}% passa | {attn:4.1f}% atenuado")

print(f"\n📐 TRANSMISSÃO TEÓRICA ACUMULADA:")
print(f"  T_total = {T_means[0]:.2f} × {T_means[1]:.2f} × {T_means[2]:.2f} × {T_means[3]:.2f} × {T_means[4]:.2f}")
print(f"  T_total = {T_theoretical*100:.3f}%")

# Fluxo real de energia
m = create_fader_mask(5, PARAMS.grid_size)
energies = model_fader.get_energy_flow(plane_wave * m)
E0 = energies[0]

print(f"\n📊 FLUXO REAL DE ENERGIA:")
print(f"  Input:     {energies[0]/E0*100:6.2f}% (referência)")
for i in range(5):
    pct = energies[i+1]/E0*100
    bar = "█" * int(pct/5) + "░" * (20 - int(pct/5))
    print(f"  Após L{i+1}:  [{bar}] {pct:5.2f}%")

final_pct = energies[-1]/E0*100
print(f"\n  ⚠️ Energia transmitida final: {final_pct:.2f}%")
print(f"     → {100-final_pct:.1f}% foi ATENUADO pelas máscaras DOE")
print(f"     → A saída visível é a REDISTRIBUIÇÃO do que passou")
print("═"*65)

In [ ]:
# ══════════════════════════════════════════════════════════════════
# CÉLULA 8: MAPAS DE TRANSMISSÃO
# ══════════════════════════════════════════════════════════════════

print("\n" + "═"*65)
print("🔍 MAPAS DE TRANSMISSÃO: O QUE CADA CAMADA APRENDEU")
print("═"*65)
print("  BRANCO = luz passa (T→1) | PRETO = luz bloqueada (T→0)")
print("═"*65)

T_maps = model_fader.get_transmission_maps()
T_means = model_fader.get_transmissions()

fig, axes = plt.subplots(1, 5, figsize=(15, 3.5))
fig.patch.set_facecolor('#0a0a0c')

for i, (ax, T_map, T_mean) in enumerate(zip(axes, T_maps, T_means)):
    im = ax.imshow(T_map, cmap='gray', vmin=0, vmax=1)
    ax.set_title(f'Camada {i+1}\nT={T_mean*100:.1f}%', color='white', fontsize=11, fontweight='bold')
    ax.axis('off')

    # Borda colorida indicando nível de atenuação
    if T_mean > 0.5:
        color = '#4CAF50'  # Verde = baixa atenuação
        status = 'baixa aten.'
    elif T_mean > 0.25:
        color = '#FF9800'  # Laranja = média atenuação
        status = 'média aten.'
    else:
        color = '#F44336'  # Vermelho = alta atenuação
        status = 'alta aten.'

    for spine in ax.spines.values():
        spine.set_edgecolor(color)
        spine.set_linewidth(3)
    ax.set_xlabel(status, color=color, fontsize=10, fontweight='bold')

plt.tight_layout()
plt.savefig('./optical_fader/transmission_maps.png', dpi=150, facecolor='#0a0a0c', bbox_inches='tight')
plt.show()


# Nova célula: Mapas de Calor de Transmissão (Azul a Amarelo)
print("\n" + "═"*70)
print("🔥 MAPAS DE CALOR DE TRANSMISSÃO - FADER COUNT")
print("═"*70)
print("  AZUL (Opaco/Baixa Transmissão) → AMARELO (Transparente/Alta Transmissão)")
print("═"*70)

fig_heatmap, axes_heatmap = plt.subplots(1, 5, figsize=(18, 4))
fig_heatmap.patch.set_facecolor('white') # Fundo branco para o gráfico

for i, (ax_h, T_map_h, T_mean_h) in enumerate(zip(axes_heatmap, T_maps, T_means)):
    im_h = ax_h.imshow(T_map_h, cmap='viridis_r', vmin=0, vmax=1) # Viridis_r: Amarelo (alto) -> Azul (baixo)
    ax_h.set_title(f'Camada {i+1}\nT={T_mean_h*100:.1f}%', color='black', fontsize=12, fontweight='bold')
    ax_h.axis('off')

plt.suptitle('Transmissão por Camada (Fader Count)', fontsize=16, fontweight='bold', color='black', y=1.05)
plt.tight_layout(rect=[0, 0.03, 0.9, 0.95]) # Ajuste para o suptitle e colorbar - deixando espaço para a colorbar

cbar_ax = fig_heatmap.add_axes([0.91, 0.15, 0.02, 0.7]) # [left, bottom, width, height] - reposicionada
cbar = fig_heatmap.colorbar(im_h, cax=cbar_ax)
cbar.set_label('Transmissão', rotation=270, labelpad=15, color='black', fontsize=12)
cbar.ax.tick_params(colors='black')

plt.savefig('./optical_fader/transmission_heatmaps_fader.png', dpi=150, facecolor='white', bbox_inches='tight')
plt.show()

In [ ]:
# ══════════════════════════════════════════════════════════════════
# CÉLULA 9: FUNÇÕES DE VISUALIZAÇÃO (V8 Style)
# ══════════════════════════════════════════════════════════════════

def save_gif_gray(data, fn, fps=5):
    """Salva GIF em escala de cinza (branco=alta intensidade, preto=baixa)."""
    frames = [((d/(d.max()+1e-8))*255).astype(np.uint8) for d in data]
    imageio.mimsave(fn, frames, fps=fps, loop=0)
    return fn

def save_gif_inferno(data, fn, fps=5):
    """Salva GIF com paleta inferno (preto→vermelho→amarelo→branco)."""
    frames = []
    for d in data:
        n = d / (d.max() + 1e-8)
        r = np.clip(n * 3, 0, 1) * 255
        g = np.clip((n - 0.35) * 4, 0, 1) * 255
        b = np.clip((n - 0.75) * 10, 0, 1) * 255
        frames.append(np.stack([r, g, b], -1).astype(np.uint8))
    imageio.mimsave(fn, frames, fps=fps, loop=0)
    return fn

def save_gif_hsv(data, fn, fps=5):
    """Salva GIF com codificação HSV para fase óptica."""
    frames = []
    for ph in data:
        pn = (ph + np.pi) / (2 * np.pi)
        c = np.ones_like(pn)
        x = c * (1 - np.abs((pn * 6) % 2 - 1))
        r, g, b = np.zeros_like(pn), np.zeros_like(pn), np.zeros_like(pn)
        h = (pn * 6).astype(int) % 6
        for hi in range(6):
            m = h == hi
            if hi == 0: r[m], g[m] = c[m], x[m]
            elif hi == 1: r[m], g[m] = x[m], c[m]
            elif hi == 2: g[m], b[m] = c[m], x[m]
            elif hi == 3: g[m], b[m] = x[m], c[m]
            elif hi == 4: r[m], b[m] = x[m], c[m]
            elif hi == 5: r[m], b[m] = c[m], x[m]
        frames.append((np.stack([r, g, b], -1) * 255).astype(np.uint8))
    imageio.mimsave(fn, frames, fps=fps, loop=0)
    return fn

def get_b64(fp):
    if fp and os.path.exists(fp):
        with open(fp, 'rb') as f:
            return base64.b64encode(f.read()).decode()
    return ""

print("✅ Funções de visualização OK")

In [ ]:
# ══════════════════════════════════════════════════════════════════
# CÉLULA 10: VISUALIZAÇÃO FADER COUNT (V8 Style - 9 Estágios)
# ══════════════════════════════════════════════════════════════════

print("🎬 Gerando visualização Fader Count...")

results = {'input': [], 'target': [], 'output': []}
for i in range(1, 6): results[f'L{i}_phase'] = []

model_fader.eval()
with torch.no_grad():
    for f in range(11):
        if f == 0:
            m = create_fader_mask(0, PARAMS.grid_size, True)
            t = create_fader_count(PARAMS.grid_size)
        else:
            m = create_fader_mask(f-1, PARAMS.grid_size)
            t = create_7segment(f-1, PARAMS.grid_size)

        inter = model_fader.forward_with_intermediates(plane_wave * m)
        results['input'].append(m.cpu().numpy())
        results['target'].append(t.cpu().numpy())
        for i in range(1, 6):
            results[f'L{i}_phase'].append(torch.angle(inter[f'L{i}']).numpy())
        results['output'].append(inter['output'].cpu().numpy())

for k in results: results[k] = np.array(results[k])

gifs = {}
gifs['inp'] = save_gif_gray(results['input'], './optical_fader/input.gif')
gifs['tgt'] = save_gif_gray(results['target'], './optical_fader/target.gif')
for i in range(1, 6): gifs[f'L{i}'] = save_gif_hsv(results[f'L{i}_phase'], f'./optical_fader/L{i}.gif')
gifs['out'] = save_gif_inferno(results['output'], './optical_fader/output.gif')

b64 = {k: get_b64(v) for k, v in gifs.items()}

html = f'''
<style>
.c9{{display:flex;flex-wrap:wrap;justify-content:center;background:#0a0a0a;padding:15px;border-radius:12px}}
.b{{text-align:center;margin:5px;background:rgba(255,255,255,0.05);padding:8px;border-radius:8px;width:11%;min-width:85px}}
.b img{{width:100%;border-radius:4px;border:2px solid #333}}
.t{{font-size:0.7em;font-weight:bold;margin-bottom:4px;color:#fff}}
</style>
<div style="text-align:center;color:#fff;font-size:1.2em;margin:10px;font-weight:bold">🔬 FADER COUNT: 9 Estágios de Propagação</div>
<div class="c9">
<div class="b"><div class="t">INPUT</div><img src="data:image/gif;base64,{b64['inp']}"></div>
<div class="b"><div class="t" style="color:#aaa">TARGET</div><img src="data:image/gif;base64,{b64['tgt']}"></div>
<div class="b"><div class="t" style="color:#4ecdc4">L1 fase</div><img src="data:image/gif;base64,{b64['L1']}"></div>
<div class="b"><div class="t" style="color:#4ecdc4">L2 fase</div><img src="data:image/gif;base64,{b64['L2']}"></div>
<div class="b"><div class="t" style="color:#4ecdc4">L3 fase</div><img src="data:image/gif;base64,{b64['L3']}"></div>
<div class="b"><div class="t" style="color:#4ecdc4">L4 fase</div><img src="data:image/gif;base64,{b64['L4']}"></div>
<div class="b"><div class="t" style="color:#4ecdc4">L5 fase</div><img src="data:image/gif;base64,{b64['L5']}"></div>
<div class="b"><div class="t" style="color:#ffe66d">OUTPUT</div><img src="data:image/gif;base64,{b64['out']}"></div>
</div>
'''
display(HTML(html))
print("✅ Fader Count OK")

In [ ]:
# ══════════════════════════════════════════════════════════════════
# CÉLULA 11: MÉTRICAS FADER COUNT
# ══════════════════════════════════════════════════════════════════

print("📐 Métricas Fader Count...")
ssim_f, psnr_f, mse_f = [], [], []
model_fader.eval()
with torch.no_grad():
    for i in range(11):
        inp = torch.from_numpy(results['input'][i]).to(DEVICE, dtype=torch.complex64)
        out = model_fader(plane_wave * inp).cpu().numpy()
        tgt = results['target'][i]
        on = np.clip((out - out.min()) / (out.max() - out.min() + 1e-8), 0, 1)
        tn = np.clip((tgt - tgt.min()) / (tgt.max() - tgt.min() + 1e-8), 0, 1)
        ssim_f.append(ssim(tn, on, data_range=1.0))
        psnr_f.append(psnr(tn, on, data_range=1.0))
        mse_f.append(((tn - on)**2).mean())

fig, ax = plt.subplots(1, 3, figsize=(13, 3.5))
fig.patch.set_facecolor('#0a0a0c')
for a, d, c, t in zip(ax, [ssim_f, psnr_f, mse_f], ['#FDD849', '#FF6B6B', '#6BCB77'], ['SSIM', 'PSNR (dB)', 'MSE']):
    a.plot(d, color=c, lw=2, marker='o', ms=5)
    a.fill_between(range(len(d)), d, alpha=0.2, color=c)
    a.set_title(f'{t}: {np.mean(d):.4f}', fontsize=10, fontweight='bold', color='white')
    a.set_facecolor('#0a0a0c'); a.tick_params(colors='white'); a.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('./optical_fader/metrics.png', dpi=150, facecolor='#0a0a0c', bbox_inches='tight')
plt.show()
print(f"✅ SSIM:{np.mean(ssim_f):.4f} | PSNR:{np.mean(psnr_f):.2f}dB | MSE:{np.mean(mse_f):.6f}")

---
# 🎬 PROPOSTA B: MICHAEL JACKSON (52 frames)

### ⚡ Por que usar Target INVERTIDO?

```
┌────────────────────────────────────────────────────────────────┐
│  PROBLEMA: GIF original tem MJ CLARO (93%) sobre fundo ESCURO │
│            → Quase toda a energia está na figura              │
│            → Para criar SILHUETA escura, precisamos BLOQUEAR  │
│              93% da luz (muito difícil!)                      │
├────────────────────────────────────────────────────────────────┤
│  SOLUÇÃO: Inverter o target!                                  │
│                                                                │
│  Original (93% claro):     Target Invertido (7% claro):       │
│  ┌──────────────┐         ┌──────────────┐                    │
│  │ ⬜ MJ claro  │   →     │ ⬛ Silhueta  │                    │
│  │ ⬛ Fundo esc │   →     │ ⬜ Fundo cla │                    │
│  └──────────────┘         └──────────────┘                    │
│                                                                │
│  💡 Agora a tarefa é ILUMINAR o fundo (fácil!) e              │
│     BLOQUEAR apenas 7% para criar a silhueta                  │
└────────────────────────────────────────────────────────────────┘
```

In [ ]:
# ══════════════════════════════════════════════════════════════════
# CÉLULA 12: DADOS MICHAEL JACKSON (V8 Style)
# ══════════════════════════════════════════════════════════════════

print("\n" + "═"*60)
print("🎬 MICHAEL JACKSON: Preparação (Estilo Negativo/Fundo Branco)")
print("═"*60)

N, NF = PARAMS.grid_size, 52
qs, fh, bp = int(0.2*N), int(0.2*N), 4
sf = N / NF

src = []
try:
    if os.path.exists('michael-jackson.gif'):
        im = Image.open('michael-jackson.gif')
        for i in range(NF):
            im.seek(int(i / (NF-1) * (im.n_frames-1)))
            frame = np.array(im.copy().convert('L').resize((N, N)), dtype=np.float32) / 255.0
            src.append(frame)
        print(f"✓ GIF carregado: {im.n_frames} frames")
    else:
        raise FileNotFoundError
except:
    print("⚠️ GIF não encontrado - usando padrão sintético")
    for i in range(NF):
        f = np.zeros((N, N), dtype=np.float32)
        y, x = np.ogrid[:N, :N]
        pos = int(i / NF * (N - 80))
        f[(x - N//2)**2 + (y - 80 - pos)**2 <= 40**2] = 0.9
        src.append(f)

mj_inp, mj_tgt = [], []
for i in range(NF):
    # INPUT (Fader)
    inp = np.ones((N, N), dtype=np.float32)
    fy = N - fh
    cx = int(i * sf)
    inp[fy:N, :bp] = inp[fy:N, N-bp:N] = inp[fy:fy+bp, :] = 0
    xe = min(cx + qs, N)
    if xe > cx:
        inp[fy+bp+2:N-2, cx:xe] = 0

    # TARGET (Fundo Branco + MJ Invertido)
    tgt = np.clip(1.0 - src[i], 0, 1)

    # TIME BAR
    bh = int(0.08 * N)
    if xe > cx:
        tgt[:bh, cx:xe] = 0.0

    mj_inp.append(torch.tensor(inp, dtype=torch.float32))
    mj_tgt.append(torch.tensor(tgt, dtype=torch.float32))

mj_inp = torch.stack(mj_inp).to(DEVICE)
mj_tgt = torch.stack(mj_tgt).to(DEVICE)

e_in = mj_inp.mean().item()
e_tgt = mj_tgt.mean().item()

print(f"\n📊 ANÁLISE DE ENERGIA (Desafiador!):")
print(f"  Input Médio:  {e_in*100:.1f}% luz (Fresta)")
print(f"  Target Médio: {e_tgt*100:.1f}% luz (Fundo Branco)")
print(f"\nInput: {mj_inp.shape} | Target: {mj_tgt.shape}")
print("═"*60)

In [ ]:
# ══════════════════════════════════════════════════════════════════
# CÉLULA 13: TREINAMENTO MJ (V8 Style + V18 Metrics)
# ══════════════════════════════════════════════════════════════════

model_mj = D2NN(PARAMS).to(DEVICE)
analyzer_mj = PhysicalPowerAnalyzer(model_mj, PARAMS)

print(f"📦 Modelo MJ: {sum(p.numel() for p in model_mj.parameters()):,} params")

print("\n" + "═"*70)
print("🔥 TREINAMENTO: MICHAEL JACKSON")
print("═"*70)
print("  ✅ Energy Scaling (V8): sc = target.mean() / output.mean()")
print(f"  ✅ Laser power: {PARAMS.laser_power_mW} mW")
print("═"*70)

opt = optim.AdamW(model_mj.parameters(), lr=0.02, weight_decay=1e-5)
sch = optim.lr_scheduler.ReduceLROnPlateau(opt, factor=0.5, patience=100)
crit = nn.MSELoss()
scaler = GradScaler()
hist_mj = []
EPOCHS = 3000
t0 = time.time()

for e in range(EPOCHS):
    model_mj.train()
    tot = 0
    for i in range(len(mj_inp)):
        opt.zero_grad()
        with autocast():
            out = model_mj(plane_wave * mj_inp[i])
            sc = mj_tgt[i].mean() / (out.mean() + 1e-8)  # Energy Scaling V8!
            loss = crit(out * sc, mj_tgt[i])
        scaler.scale(loss).backward()
        scaler.step(opt)
        scaler.update()
        tot += loss.item()
    avg = tot / len(mj_inp)
    hist_mj.append(avg)
    sch.step(avg)

    if e % 400 == 0 or e == EPOCHS - 1:
        T = model_mj.get_transmissions()
        # Medições em mW (V18)
        mid_idx = len(mj_inp) // 2
        test_input = plane_wave * mj_inp[mid_idx]
        powers = analyzer_mj.calculate_power_mW(test_input)
        print(f"\nEpoch {e}/{EPOCHS}:")
        print(f"  Loss: {avg:.6f}")
        print(f"  T: [{T[0]:.2f},{T[1]:.2f},{T[2]:.2f},{T[3]:.2f},{T[4]:.2f}]")
        print(f"  Detector Power: {powers['output_mW']:.4f} mW")
        print(f"  Efficiency: {powers['efficiency_%']:.3f}%")
        print(f"  SNR (1ms): {powers['SNR_1ms']:.1f}")

        # Export DOEs
        analyzer_mj.export_does_16bit('optical_mj', f'epoch_{e:04d}')

time_mj = time.time() - t0
print(f"\n✅ {int(time_mj//60)}m{int(time_mj%60)}s | Loss: {hist_mj[0]:.6f} → {hist_mj[-1]:.6f}")

# Análise final
mid_idx = len(mj_inp) // 2
test_input = plane_wave * mj_inp[mid_idx]
analyzer_mj.print_power_summary(test_input, "MJ FINAL ANALYSIS")

# Export DOEs finais
analyzer_mj.export_does_16bit('optical_mj', 'final')

# Salvar modelo
torch.save(model_mj.state_dict(), 'optical_mj/model_mj_v19.pth')
print("💾 Model saved: optical_mj/model_mj_v19.pth")

In [ ]:
# ══════════════════════════════════════════════════════════════════
# CÉLULA 14: ANÁLISE DE ENERGIA MJ
# ══════════════════════════════════════════════════════════════════

print("\n" + "═"*65)
print("⚡ ANÁLISE DE ENERGIA MJ: ATENUAÇÃO vs REDISTRIBUIÇÃO")
print("═"*65)

T_means_mj = model_mj.get_transmissions()
T_theoretical_mj = model_mj.get_theoretical_transmission()

print("\n📊 TRANSMISSÃO POR CAMADA (MJ):")
for i, t in enumerate(T_means_mj):
    bar = "█" * int(t * 20) + "░" * (20 - int(t * 20))
    attn = (1 - t) * 100
    print(f"  L{i+1}: [{bar}] {t*100:5.1f}% passa | {attn:4.1f}% atenuado")

print(f"\n📐 TRANSMISSÃO TEÓRICA ACUMULADA:")
print(f"  T_total = {T_theoretical_mj*100:.3f}%")

# Fluxo real de energia
energies_mj = model_mj.get_energy_flow(plane_wave * mj_inp[25])
E0_mj = energies_mj[0]

print(f"\n📊 FLUXO REAL DE ENERGIA (frame 25):")
for i in range(6):
    pct = energies_mj[i]/E0_mj*100
    bar = "█" * int(pct/5) + "░" * (20 - int(pct/5))
    stage = "Input" if i == 0 else f"Após L{i}"
    print(f"  {stage:8}: [{bar}] {pct:5.2f}%")

final_pct_mj = energies_mj[-1]/E0_mj*100
print(f"\n  ⚠️ Energia transmitida final: {final_pct_mj:.2f}%")
print("═"*65)

# 📊 Análise da Evolução das Especificações DOE

In [ ]:
# cell_7_energy_analysis

import re

# Data structures to store specs over epochs
fader_specs_history = {'height_max': {i: [] for i in range(PARAMS.num_layers)},
                       'transmission_mean': {i: [] for i in range(PARAMS.num_layers)}}

mj_specs_history = {'height_max': {i: [] for i in range(PARAMS.num_layers)},
                    'transmission_mean': {i: [] for i in range(PARAMS.num_layers)}}

def parse_specs_file(filepath):
    """Parses a fabrication specs file and extracts max height and mean transmission per layer."""
    height_max_per_layer = {}
    transmission_mean_per_layer = {}

    with open(filepath, 'r', encoding='utf-8') as f:
        content = f.read()

    # Regex to find layer data
    layer_blocks = re.findall(r"Layer (\d+):\n\s+Phase Height: ([\d.-]+)-([\d.-]+) µm\n\s+Amplitude: T=([\d.-]+)% \(σ=[\d.-]+%\)", content)

    for block in layer_blocks:
        layer_idx = int(block[0]) - 1 # 0-indexed
        max_height = float(block[2])
        mean_transmission = float(block[3]) / 100.0 # Convert to [0,1]

        height_max_per_layer[layer_idx] = max_height
        transmission_mean_per_layer[layer_idx] = mean_transmission
    return height_max_per_layer, transmission_mean_per_layer

# Collect data from fader fabrication spec files
fader_epochs = [0, 500, 1000, 1500, 2000] # Adjust based on actual epochs saved
for epoch in fader_epochs:
    filepath = f'optical_fader/epoch_{epoch:04d}_fabrication_specs.txt'
    if os.path.exists(filepath):
        h_max, t_mean = parse_specs_file(filepath)
        for layer_idx in range(PARAMS.num_layers):
            if layer_idx in h_max:
                fader_specs_history['height_max'][layer_idx].append(h_max[layer_idx])
                fader_specs_history['transmission_mean'][layer_idx].append(t_mean[layer_idx])

# Collect data from mj fabrication spec files
mj_epochs = [0, 400, 800, 1200, 1600, 1999] # Adjust based on actual epochs saved
for epoch in mj_epochs:
    filepath = f'optical_mj/epoch_{epoch:04d}_fabrication_specs.txt'
    if os.path.exists(filepath):
        h_max, t_mean = parse_specs_file(filepath)
        for layer_idx in range(PARAMS.num_layers):
            if layer_idx in h_max:
                mj_specs_history['height_max'][layer_idx].append(h_max[layer_idx])
                mj_specs_history['transmission_mean'][layer_idx].append(t_mean[layer_idx])

# Plotting function
def plot_doe_specs_evolution(specs_history, epochs, title, output_filename):
    fig, ax1 = plt.subplots(figsize=(10, 6))
    fig.patch.set_facecolor('white')

    colors_h = plt.cm.Blues(np.linspace(0.4, 1, PARAMS.num_layers))
    colors_t = plt.cm.Oranges(np.linspace(0.4, 1, PARAMS.num_layers))

    ax1.set_xlabel('Epoch', color='black')
    ax1.set_ylabel('Max Phase Height (µm)', color='blue')
    ax1.tick_params(axis='y', labelcolor='blue')
    ax1.tick_params(axis='x', labelcolor='black')

    ax2 = ax1.twinx()
    ax2.set_ylabel('Mean Transmission', color='orange')
    ax2.tick_params(axis='y', labelcolor='orange')

    lines = []
    for layer_idx in range(PARAMS.num_layers):
        # Plot Max Phase Height
        line1, = ax1.plot(epochs[:len(specs_history['height_max'][layer_idx])],
                          specs_history['height_max'][layer_idx],
                          color=colors_h[layer_idx], linestyle='-', marker='o',
                          label=f'L{layer_idx+1} Max Height')
        lines.append(line1)

        # Plot Mean Transmission
        line2, = ax2.plot(epochs[:len(specs_history['transmission_mean'][layer_idx])],
                          specs_history['transmission_mean'][layer_idx],
                          color=colors_t[layer_idx], linestyle='--', marker='x',
                          label=f'L{layer_idx+1} Mean Transmission')
        lines.append(line2)

    ax1.set_title(title, color='black', fontsize=16)
    ax1.grid(True, linestyle='--', alpha=0.7)

    # Create a single legend for both axes
    labs = [l.get_label() for l in lines]
    ax1.legend(lines, labs, loc='center left', bbox_to_anchor=(1.1, 0.5), fontsize=9)

    plt.tight_layout()
    plt.savefig(output_filename, dpi=150, facecolor='white')
    plt.show()

# Plot for Fader Count
plot_doe_specs_evolution(fader_specs_history, fader_epochs,
                         'Fader Count DOE Specs Evolution', './fader_doe_specs_evolution.png')

# Plot for Michael Jackson
plot_doe_specs_evolution(mj_specs_history, mj_epochs,
                         'Michael Jackson DOE Specs Evolution', './mj_doe_specs_evolution.png')

In [ ]:
# ══════════════════════════════════════════════════════════════════
# CÉLULA 15: VISUALIZAÇÃO MJ (V8 Style - 9 Estágios)
# ══════════════════════════════════════════════════════════════════

print("🎬 Gerando visualização MJ (9 estágios)...")

mj_results = {'input': [], 'target': [], 'output': []}
for i in range(1, 6): mj_results[f'L{i}_phase'] = []

model_mj.eval()
with torch.no_grad():
    for i in range(len(mj_inp)):
        current_input = mj_inp[i]
        current_target = mj_tgt[i]

        inter = model_mj.forward_with_intermediates(plane_wave * current_input)
        mj_results['input'].append(current_input.cpu().numpy())
        mj_results['target'].append(current_target.cpu().numpy())
        for l_idx in range(1, 6):
            mj_results[f'L{l_idx}_phase'].append(torch.angle(inter[f'L{l_idx}']).numpy())

        output = inter['output']
        mj_results['output'].append((output / (output.max() + 1e-8)).cpu().numpy())

for k in mj_results: mj_results[k] = np.array(mj_results[k])

# Criar GIFs
gifs_mj = {}
gifs_mj['inp'] = save_gif_gray(mj_results['input'], './optical_mj/input.gif', fps=10)
gifs_mj['tgt'] = save_gif_gray(mj_results['target'], './optical_mj/target.gif', fps=10)
for i in range(1, 6):
    gifs_mj[f'L{i}'] = save_gif_hsv(mj_results[f'L{i}_phase'], f'./optical_mj/L{i}.gif', fps=10)
gifs_mj['out'] = save_gif_inferno(mj_results['output'], './optical_mj/output.gif', fps=10)

b64mj = {k: get_b64(v) for k, v in gifs_mj.items()}

html_mj = f'''
<style>
.c9{{display:flex;flex-wrap:wrap;justify-content:center;background:#0a0a0a;padding:15px;border-radius:12px}}
.b{{text-align:center;margin:5px;background:rgba(255,255,255,0.05);padding:8px;border-radius:8px;width:11%;min-width:85px}}
.b img{{width:100%;border-radius:4px;border:2px solid #333}}
.t{{font-size:0.7em;font-weight:bold;margin-bottom:4px;color:#fff}}
</style>
<div style="text-align:center;color:#fff;font-size:1.2em;margin:10px;font-weight:bold">🎬 MICHAEL JACKSON: 9 Estágios de Propagação</div>
<div class="c9">
<div class="b"><div class="t">INPUT</div><img src="data:image/gif;base64,{b64mj['inp']}"></div>
<div class="b"><div class="t" style="color:#aaa">TARGET</div><img src="data:image/gif;base64,{b64mj['tgt']}"></div>
<div class="b"><div class="t" style="color:#4ecdc4">L1 fase</div><img src="data:image/gif;base64,{b64mj['L1']}"></div>
<div class="b"><div class="t" style="color:#4ecdc4">L2 fase</div><img src="data:image/gif;base64,{b64mj['L2']}"></div>
<div class="b"><div class="t" style="color:#4ecdc4">L3 fase</div><img src="data:image/gif;base64,{b64mj['L3']}"></div>
<div class="b"><div class="t" style="color:#4ecdc4">L4 fase</div><img src="data:image/gif;base64,{b64mj['L4']}"></div>
<div class="b"><div class="t" style="color:#4ecdc4">L5 fase</div><img src="data:image/gif;base64,{b64mj['L5']}"></div>
<div class="b"><div class="t" style="color:#ffe66d">OUTPUT</div><img src="data:image/gif;base64,{b64mj['out']}"></div>
</div>
'''
display(HTML(html_mj))
print("✅ MJ OK")

In [ ]:
# ══════════════════════════════════════════════════════════════════
# CÉLULA 16: MÉTRICAS MJ
# ══════════════════════════════════════════════════════════════════

print("📐 Métricas MJ...")
ssim_mj, psnr_mj, mse_mj = [], [], []
model_mj.eval()
with torch.no_grad():
    for i in range(len(mj_inp)):
        out = model_mj(plane_wave * mj_inp[i]).cpu().numpy()
        tgt = mj_tgt[i].cpu().numpy()
        on = np.clip((out - out.min()) / (out.max() - out.min() + 1e-8), 0, 1)
        ssim_mj.append(ssim(tgt, on, data_range=1.0))
        psnr_mj.append(psnr(tgt, on, data_range=1.0))
        mse_mj.append(((tgt - on)**2).mean())

fig, ax = plt.subplots(1, 3, figsize=(13, 3.5))
fig.patch.set_facecolor('#0a0a0c')
for a, d, c, t in zip(ax, [ssim_mj, psnr_mj, mse_mj], ['#FDD849', '#FF6B6B', '#6BCB77'], ['SSIM', 'PSNR (dB)', 'MSE']):
    a.plot(d, color=c, lw=1.5, marker='o', ms=2, alpha=0.7)
    a.fill_between(range(len(d)), d, alpha=0.2, color=c)
    a.set_title(f'{t}: {np.mean(d):.4f}', fontsize=10, fontweight='bold', color='white')
    a.set_facecolor('#0a0a0c'); a.tick_params(colors='white'); a.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('./optical_mj/metrics.png', dpi=150, facecolor='#0a0a0c', bbox_inches='tight')
plt.show()
print(f"✅ SSIM:{np.mean(ssim_mj):.4f} | PSNR:{np.mean(psnr_mj):.2f}dB | MSE:{np.mean(mse_mj):.6f}")

In [ ]:
# ══════════════════════════════════════════════════════════════════
# CÉLULA 17: COMPARAÇÃO DE MAPAS DE TRANSMISSÃO
# ══════════════════════════════════════════════════════════════════

print("\n" + "═"*65)
print("🔍 COMPARAÇÃO: Mapas de Transmissão Fader vs MJ")
print("═"*65)

T_maps_mj = model_mj.get_transmission_maps()
T_means_mj = model_mj.get_transmissions()
T_maps_f = model_fader.get_transmission_maps()
T_means_f = model_fader.get_transmissions()

fig, axes = plt.subplots(2, 5, figsize=(15, 6))
fig.patch.set_facecolor('#0a0a0c')

for row, (maps, means, name, color) in enumerate([
    (T_maps_f, T_means_f, 'Fader', '#FF4500'),
    (T_maps_mj, T_means_mj, 'MJ', '#4169E1')
]):
    for i, (ax, T_map, T_mean) in enumerate(zip(axes[row], maps, means)):
        ax.imshow(T_map, cmap='gray', vmin=0, vmax=1)
        ax.set_title(f'L{i+1}: {T_mean*100:.0f}%', color='white', fontsize=10)
        ax.axis('off')
        for spine in ax.spines.values():
            spine.set_edgecolor(color)
            spine.set_linewidth(2)
    axes[row, 0].set_ylabel(name, color=color, fontsize=12, fontweight='bold', rotation=0, labelpad=30)

plt.tight_layout()
plt.savefig('./transmission_comparison.png', dpi=150, facecolor='#0a0a0c', bbox_inches='tight')
plt.show()

print("\n📊 COMPARAÇÃO DE TRANSMISSÃO:")
print("─" * 60)
for i in range(5):
    bf = "█" * int(T_means_f[i] * 15) + "░" * (15 - int(T_means_f[i] * 15))
    bm = "█" * int(T_means_mj[i] * 15) + "░" * (15 - int(T_means_mj[i] * 15))
    print(f"  L{i+1}: Fader [{bf}] {T_means_f[i]*100:4.0f}% | MJ [{bm}] {T_means_mj[i]*100:4.0f}%")

T_total_f = model_fader.get_theoretical_transmission()
T_total_mj = model_mj.get_theoretical_transmission()
print(f"\n  📐 Transmissão total: Fader={T_total_f*100:.2f}% | MJ={T_total_mj*100:.2f}%")

In [ ]:
# ══════════════════════════════════════════════════════════════════
# CÉLULA 18: ANÁLISE COMPARATIVA FINAL
# ══════════════════════════════════════════════════════════════════

print("\n" + "═"*75)
print("📊 ANÁLISE COMPARATIVA FINAL - D²NN v19 CONSOLIDADO")
print("═"*75)

# Potência final
powers_fader = analyzer_fader.calculate_power_mW(plane_wave * create_fader_mask(5, PARAMS.grid_size))
powers_mj = analyzer_mj.calculate_power_mW(plane_wave * mj_inp[25])

print(f"""
┌────────────────────┬───────────────┬───────────────────┐
│ MÉTRICA            │ FADER COUNT   │ MICHAEL JACKSON   │
├────────────────────┼───────────────┼───────────────────┤
│ Frames             │      11       │        52         │
│ Tempo              │   {time_fader:>6.0f}s     │     {time_mj:>6.0f}s      │
│ SSIM               │    {np.mean(ssim_f):>6.4f}    │      {np.mean(ssim_mj):>6.4f}      │
│ PSNR (dB)          │    {np.mean(psnr_f):>6.2f}     │      {np.mean(psnr_mj):>6.2f}       │
│ MSE                │  {np.mean(mse_f):>8.6f}  │    {np.mean(mse_mj):>8.6f}    │
│ T_total            │    {T_total_f*100:>5.2f}%    │      {T_total_mj*100:>5.2f}%      │
├────────────────────┼───────────────┼───────────────────┤
│ Potência (mW)      │    {powers_fader['output_mW']:>6.4f}   │      {powers_mj['output_mW']:>6.4f}     │
│ Eficiência (%)     │    {powers_fader['efficiency_%']:>6.3f}   │      {powers_mj['efficiency_%']:>6.3f}     │
│ SNR (1ms)          │  {powers_fader['SNR_1ms']:>9.1f}  │    {powers_mj['SNR_1ms']:>9.1f}   │
└────────────────────┴───────────────┴───────────────────┘
""")

# Fluxo de energia comparativo
print("\n⚡ FLUXO DE ENERGIA COMPARATIVO:")
m_f = create_fader_mask(5, PARAMS.grid_size)
E_fader = model_fader.get_energy_flow(plane_wave * m_f)
E_mj = model_mj.get_energy_flow(plane_wave * mj_inp[25])
E0_f, E0_m = E_fader[0], E_mj[0]

print("         Fader                    MJ")
print("─" * 55)
stages = ['Input', 'L1', 'L2', 'L3', 'L4', 'L5']
for i, stage in enumerate(stages):
    pf = E_fader[i]/E0_f*100
    pm = E_mj[i]/E0_m*100
    bf = "█" * int(pf/5) + "░" * (20 - int(pf/5))
    bm = "█" * int(pm/5) + "░" * (20 - int(pm/5))
    print(f"{stage:>6}: [{bf}] {pf:5.1f}%  [{bm}] {pm:5.1f}%")

# Curvas de convergência
fig, ax = plt.subplots(1, 2, figsize=(12, 4))
fig.patch.set_facecolor('#0a0a0c')
ax[0].plot(history_fader, color='#FF4500', lw=2); ax[0].set_yscale('log')
ax[0].set_title('Fader Count', fontsize=11, fontweight='bold', color='white')
ax[0].set_facecolor('#0a0a0c'); ax[0].tick_params(colors='white'); ax[0].grid(True, alpha=0.3)
ax[1].plot(hist_mj, color='#4169E1', lw=2); ax[1].set_yscale('log')
ax[1].set_title('Michael Jackson', fontsize=11, fontweight='bold', color='white')
ax[1].set_facecolor('#0a0a0c'); ax[1].tick_params(colors='white'); ax[1].grid(True, alpha=0.3)
plt.suptitle('📉 Convergência', fontsize=12, fontweight='bold', color='white')
plt.tight_layout();
plt.savefig('./convergence.png', dpi=150, facecolor='#0a0a0c', bbox_inches='tight')
plt.show()

# Novo gráfico de convergência solicitado (fundo branco, cores específicas)
fig_new, ax_new = plt.subplots(1, 1, figsize=(8, 5))
fig_new.patch.set_facecolor('white')

ax_new.plot(history_fader, color='red', lw=4, label='Fader Count')
ax_new.plot(hist_mj, color='blue', lw=4, label='Michael Jackson')

ax_new.set_yscale('log')
ax_new.set_title('Convergência do Treinamento', fontsize=28, fontweight='bold', color='black')
ax_new.set_xlabel('Épocas', fontsize=24, color='black')
ax_new.set_ylabel('Loss (log scale)', fontsize=24, color='black')
ax_new.set_facecolor('white')
ax_new.tick_params(colors='black')
ax_new.grid(True, alpha=0.5, color='gray')
ax_new.legend(fontsize=20, facecolor='white', framealpha=1, edgecolor='black')

plt.tight_layout()
plt.savefig('./convergence_white_background.png', dpi=150, facecolor='white', bbox_inches='tight')
plt.show()

In [ ]:
# ══════════════════════════════════════════════════════════════════
# CÉLULA 19: SUMÁRIO FINAL
# ══════════════════════════════════════════════════════════════════

print("\n" + "═"*75)
print("📁 ESTRUTURA DE ARQUIVOS GERADA - D²NN v19 CONSOLIDADO")
print("═"*75)

print("""
optical_fader/
├── input.gif                    ← Input animado (11 frames)
├── output.gif                   ← Output animado (inferno colormap)
├── target.gif                   ← Target animado
├── L1.gif ... L5.gif            ← Fases de cada camada (HSV colormap)
├── metrics.png                  ← Gráficos SSIM/PSNR/MSE
├── transmission_maps.png        ← Mapas de transmissão
├── epoch_0000_L{1-5}_phase_16bit.tiff  ← DOEs 16-bit (época 0)
├── epoch_0000_L{1-5}_amp_8bit.png
├── epoch_0000_fabrication_specs.txt
├── epoch_0500_L{1-5}_phase_16bit.tiff  ← DOEs 16-bit (época 500)
├── ...
├── final_L{1-5}_phase_16bit.tiff       ← DOEs 16-bit FINAIS
├── final_L{1-5}_amp_8bit.png
├── final_L{1-5}_height_um.npy          ← Altura em µm (NPY)
├── final_L{1-5}_transmission.npy       ← Transmissão (NPY)
├── final_fabrication_specs.txt         ← Relatório de fabricação
└── model_fader_v19.pth                 ← Modelo PyTorch

optical_mj/
└── (mesma estrutura, 52 frames)

Raiz:
├── transmission_comparison.png  ← Comparação Fader vs MJ
└── convergence.png              ← Curvas de convergência
""")

print("═"*75)
print("✨ FEATURES D²NN v19 CONSOLIDADO:")
print("═"*75)
print("""
  📊 DA V8 (Visualização):
     ✅ Energy Scaling: sc = target.mean() / output.mean()
     ✅ GIFs de alta qualidade com padrões claros
     ✅ Métricas SSIM/PSNR/MSE
     ✅ Mapas de transmissão coloridos
     ✅ Fluxo de energia visual

  🔬 DA V18 (Física):
     ✅ Parâmetros físicos completos (λ, n, laser power)
     ✅ Medições em mW durante treinamento
     ✅ Cálculo de eficiência óptica
     ✅ Estimativa de SNR (shot noise limited)
     ✅ Export DOEs 16-bit TIFF para fabricação
     ✅ Export amplitude 8-bit PNG
     ✅ Altura de fase em µm (NPY)
     ✅ Relatórios de especificação de fabricação
""")

print("═"*75)
print("📚 REFERÊNCIAS:")
print("[1] Chen et al. Nature 644, 903-910 (2025)")
print("[2] Lin et al. Science 361, 1004-1008 (2018)")
print("[3] Goodman, Introduction to Fourier Optics (2005)")
print("═"*75)
print("\n🎉 D²NN v19 CONSOLIDADO COMPLETO!")